$\newcommand{\giraffe}{\texttt{GiRaFFE}}$
$\newcommand{\gf}{\texttt{GiRaFFEFood}}$
## $\gf$: An Einstein Toolkit Initial Data Thorn for $\giraffe$

### NRPy+ Source Code for this module: [GiRaFFEFood_HO.py](../edit/GiRaFFEFood_HO.py) , which is fully documented in the [previous NRPy+ tutorial module](Tutorial-GiRaFFEFood_HO.ipynb) on using NRPy+ to construct these Exact Wald initial data as SymPy expressions.


In this part of the tutorial, we will construct an Einstein Toolkit (ETK) thorn (module) that will set up *initial data* for $\giraffe$. In a [previous tutorial module](Tutorial-MaxwellCartesian.ipynb), we used NRPy+ to contruct the SymPy expressions for plane-wave initial data. 

We will construct this thorn in two steps.

1. Call on NRPy+ to convert the SymPy expressions for the initial data into one C-code kernel.
1. Write the C code and linkages to the Einstein Toolkit infrastructure (i.e., the .ccl files) to complete this Einstein Toolkit module.

### Step 1: Call on NRPy+ to convert the SymPy expression for the Eact Wald initial data into a C-code kernel. 

After importing the core modules, we will set $\text{GridFuncMemAccess}$ to $\text{ETK}$. SymPy expressions for plane wave initial data are written inside [GiRaFFEFood_HO.py](../edit/GiRaFFEFood_HO.py), and we simply import them for use here.

In [1]:
# Step 1a: Import needed NRPy+ core modules:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import loop


# Step 1b: This is an Einstein Toolkit (ETK) thorn. Here we
#          tell NRPy+ that gridfunction memory access will 
#          therefore be in the "ETK" style.
par.set_parval_from_str("grid::GridFuncMemAccess","ETK")
#Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

import GiRaFFEFood_HO as gfho
gfho.GiRaFFEFood_HO()

# Step 2: Create the C code output kernel.
GiRaFFEFood_to_print = [\
                        lhrh(lhs=gri.gfaccess("out_gfs","AD0"),rhs=gfho.AD[0]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","AD1"),rhs=gfho.AD[1]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","AD2"),rhs=gfho.AD[2]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=gfho.BU[0]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=gfho.BU[1]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=gfho.BU[2]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU0"),rhs=gfho.ValenciavU[0]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU1"),rhs=gfho.ValenciavU[1]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU2"),rhs=gfho.ValenciavU[2]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","StildeD0"),rhs=gfho.StildeD[0]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","StildeD1"),rhs=gfho.StildeD[1]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","StildeD2"),rhs=gfho.StildeD[2]),\
                        ]

GiRaFFEFood_CKernel = fin.FD_outputC("returnstring",GiRaFFEFood_to_print)

GiRaFFEFood_looped = loop.loop(["i2","i1","i0"],["0","0","0"],["cctk_lsh[2]","cctk_lsh[1]","cctk_lsh[0]"],\
                               ["1","1","1"],["#pragma omp parallel for","",""],"",\
                               GiRaFFEFood_CKernel.replace("time","cctk_time"))

# Step 3: Create directories for the thorn if they don't exist.
!mkdir GiRaFFEFood     2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.
!mkdir GiRaFFEFood/src 2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.

# Step 4: Write the C code kernel to file.
with open("GiRaFFEFood/src/GiRaFFEFood_ExactWald.h", "w") as file:
    file.write(str(GiRaFFEFood_looped))


### Step 2: Interfacing with the Einstein Toolkit

#### Step 2a: Constructing the Einstein Toolkit C-code calling functions that include the C code kernels.

We will write another C file with the functions we need here.

### Step 2b: CCL files - Define how this module interacts and interfaces with the larger Einstein Toolkit infrastructure

Writing a module ("thorn") within the Einstein Toolkit requires that three "ccl" files be constructed, all in the root directory of the thorn:

1. $\text{interface.ccl}$: defines the gridfunction groups needed, and provides keywords denoting what this thorn provides and what it should inherit from other thorns. Specifically, this file governs the interaction between this thorn and others; more information can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-260000C2.2). 
With "implements", we give our thorn its unique name. By "inheriting" other thorns, we tell the Toolkit that we will rely on variables that exist and are declared "public" within those functions.

2. $\text{param.ccl}$: specifies free parameters within the thorn, enabling them to be set at runtime. It is required to provide allowed ranges and default values for each parameter. More information on this file's syntax can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-265000C2.3).

3. $\text{schedule.ccl}$: allocates storage for gridfunctions, defines how the thorn's functions should be scheduled in a broader simulation, and specifies the regions of memory written to or read from gridfunctions. $\text{schedule.ccl}$'s official documentation may be found [here](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-268000C2.4). 

We specify here the standardized ETK "scheduling bins" in which we want each of our thorn's functions to run.

#### Step 2c: Add the C code to the Einstein Toolkit compilation list.

We will also need $\text{make.code.defn}$, which indicates the list of files that need to be compiled. This thorn only has the one C file to compile.